In [1]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from tabular_benchmark.utils.misc import set_seeds
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Comparision between GBDT model from sklearn and tabular_benchmark

## Sklearn

### Load and Split dataset

This dataset was derived from the 1990 U.S. census, using one row per census block group. A block group is the smallest geographical unit for which the U.S. Census Bureau publishes sample data (a block group typically has a population of 600 to 3,000 people).
For more information:
https://scikit-learn.org/stable/datasets/real_world.html#california-housing-dataset

In [2]:
dataset = datasets.fetch_california_housing()
dataset

8 numeric, predictive attributes and the target
- MedInc median income in block group
- HouseAge median house age in block group
- AveRooms average number of rooms per household
- AveBedrms average number of bedrooms per household
- Population block group population
- AveOccup average number of household members
- Latitude block group latitude
- Longitude block group longitude

In [3]:
raw_X = dataset['data']
raw_X

20640 samples

In [4]:
raw_X.shape

The target variable is the median house value for California districts, expressed in hundreds of thousands of dollars ($100,000). Therefore, we are dealing with a **regression** problem

In [5]:
raw_y = dataset['target']
raw_y 

In [6]:
plt.hist(raw_y)
plt.xlabel('House Value (thousands of USD)')
plt.tight_layout()
plt.savefig('california_housing_dist')

### Load default model

In [7]:
model = GradientBoostingRegressor()  # classic GBDT from sklearn for regression

### Preprocess dataset

We need to do at least some preprocessing, starting by spliting the data in train, validation and test. We will also transform our data to a pandas dataframe to facilitate our work.

In [8]:
set_seeds(42)  # set seeds of python, numpy and pytorch altogheter
raw_X = pd.DataFrame(raw_X)
raw_y = pd.DataFrame(raw_y)
pct_test = 0.2  # pct of total data
pct_valid = 0.1 * (1 - pct_test)  # pct of total data
seed = 42  # reproducibility
train_data, X_test, train_target, y_test = train_test_split(raw_X, raw_y, test_size=pct_test, random_state=seed)
X_train, X_valid, y_train, y_valid = train_test_split(train_data, train_target, test_size=pct_valid, random_state=seed)

In [9]:
X_train

We need to ensure that there is no missing data. We will replace any missing data by the feature median. 

In [10]:
median_nan = X_train.median()
X_train = X_train.fillna(median_nan)
X_valid = X_valid.fillna(median_nan)
X_test = X_test.fillna(median_nan)

We know that we dot not have any categorical data, if we had, we would also need to ensure that there is no missing data (replacing missing values for the mode of the feature, for example) and we would need to encode the categorical features using some strategy (ordinal encoding, one-hot encoding etc).

We will now cast our data to a standard type.

In [11]:
X_train = X_train.astype(np.float32)
X_valid = X_valid.astype(np.float32)
X_test = X_test.astype(np.float32)
y_train = y_train.astype(np.float32)
y_valid = y_valid.astype(np.float32)
y_test = y_test.astype(np.float32)

### Fit the model

In [12]:
y_train = np.ravel(y_train)  # sklearn expects the shape of y to be (n_samples, ) 
model.fit(X_train, y_train)

### Visualizing the result

By default the model uses the squared error

In [13]:
model.train_score_

In [14]:
fig, ax = plt.subplots()
ax.plot(model.train_score_)
ax.set_xlabel('iterations')
ax.set_ylabel('MSE')

In [15]:
y_pred = model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
results_columns = ['Model', 'RMSE']
results = pd.DataFrame(columns=results_columns)
res = ['Scikit-learn GBDT', rmse]
res_row = pd.DataFrame(dict(zip(results_columns, res)), index=[0])
results = pd.concat([results, res_row], ignore_index=True)
results

## Tabular benchmark

In [16]:
from tabular_benchmark.utils.datasets import load_dataset
from tabular_benchmark.models.xgboost import XGBoostModel
from tabular_benchmark.utils.misc import train_valid_test_split

### Load and Split dataset

In [17]:
dataset = load_dataset('california_housing')

In [18]:
raw_X = dataset['data']
raw_X

In [19]:
raw_y = dataset['target']
raw_y

Note that we are already working with DataFrames

In [20]:
task = dataset['task']
cat_features = dataset['cat_features']

In [21]:
set_seeds(42)  # set seeds of python, numpy and pytorch altogheter
pct_test = 0.2  # pct of total data
pct_valid = 0.1 * (1 - pct_test)  # pct of total data
seed = 42  # reproducibility
train_data, X_test, train_target, y_test = train_test_split(raw_X, raw_y, test_size=pct_test, random_state=seed)
X_train, X_valid, y_train, y_valid = train_test_split(train_data, train_target, test_size=pct_valid, random_state=seed)
train = [X_train, y_train]
valid = [X_valid, y_valid]
test = [X_test, y_test]
# We could run the code below, but we are using the same split as before to compare the methods
# pct_test = 0.2  # pct of total data
# pct_valid = 0.1 * (1 - pct_test)  # pct of total data
# train, valid, test = train_valid_test_split(raw_X, raw_y, pct_valid, pct_test, random_state=seed)

Each set is a list of [features, target]

In [22]:
train

### Load default model

In [23]:
model = XGBoostModel()

Note that we do not need to specify if it is a regressor or a classifier, we will automatically instanciate the right model given the task.

### Preprocess dataset

In [24]:
train, valid, test, p_cat_features, p_cat_dims = model.preprocess_dataset(train=train, task=task, cat_features=cat_features,
                                                                          valid=valid, test=test)

This function automatically treat missing values, encode categorical features, cast the variables to its correct type and normalize the features following the recommendations for each model. It also return the categorical features and the dimensions of each categorical feature, which are needed for some models.

### Fit the model

In [25]:
X_train = train[0]
y_train = train[1]
eval_sets = [train, valid]
eval_names = ['train', 'valid']

In [26]:
model.fit(X_train=X_train, y_train=y_train, task=task, eval_sets=eval_sets, eval_names=eval_names, 
          cat_features=p_cat_features, cat_dims=p_cat_dims)

### Visualizing the result

By default the model uses **root** mean squared error

In [27]:
model.evals_result_

In [28]:
fig, ax = plt.subplots()
ax.plot(model.evals_result_['train']['loss_rmse'], label='train')
ax.plot(model.evals_result_['valid']['loss_rmse'], label='valid')
ax.legend()
ax.set_xlabel('iterations')
ax.set_ylabel('RMSE')

Note that we use by default the valid set for early stopping (we could do the same with sklearn with some tweaking)

In [29]:
y_pred = model.predict(test[0])
rmse = mean_squared_error(test[1], y_pred, squared=False)
res = ['XGboost', rmse]
res_row = pd.DataFrame(dict(zip(results_columns, res)), index=[0])
results = pd.concat([results, res_row], ignore_index=True)
results

In [30]:
# Or Equivalently...
model.evaluate_set(test, 'rmse')

### Beyond the default model...tunning the hyperparameters

In [31]:
from tabular_benchmark.tunners import DEHBTunner

In [32]:
model = XGBoostModel()

We use the total training data, before spliting into the validation set, and we choose a budget_type ('iteration' or 'subsample') to perform the tunning of the hyperparameters.

In [34]:
tunner = DEHBTunner(budget_type='iteration',
                    min_budget=100, max_budget=10000, resample_strategy='k-fold_cv', k_folds=5,
                    seed_dataset=seed, seed_model=seed,
                    n_workers=5)

We run until one of the conditions is met (we should only set **one**):
- we evaluate a certain amount of models (fevals)
- we evaluate a certain amount of brackets (min budget to max budget)
- we evaluate for a certain amount of time in seconds (total_cost)

In [35]:
fevals = None
brackets = 10
total_cost = None
trajectory, runtime, history = tunner.run(model.__class__, train_data, train_target, task, cat_features,
                                          fevals=fevals, brackets=brackets, total_cost=total_cost,
                                          verbose=False, save_intermediate=False)

In [ ]:
tunner.tunner.get_incumbents()

In [ ]:
model = XGBoostModel(**tunner.tunner.get_incumbents()[0])

In [ ]:
model.fit(X_train=X_train, y_train=y_train, task=task, eval_sets=eval_sets, eval_names=eval_names, 
          cat_features=p_cat_features, cat_dims=p_cat_dims)

In [ ]:
fig, ax = plt.subplots()
ax.plot(model.evals_result_['train']['rmse'], label='train')
ax.plot(model.evals_result_['valid']['rmse'], label='valid')
ax.legend()
ax.set_xlabel('iterations')
ax.set_ylabel('RMSE')

In [ ]:
y_pred = model.predict(test[0])
rmse = mean_squared_error(test[1], y_pred, squared=False)
res = ['XGboost Tunned', rmse]
res_row = pd.DataFrame(dict(zip(results_columns, res)), index=[0])
results = pd.concat([results, res_row], ignore_index=True)
results